In [1]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import requests
from io import StringIO
from os import listdir
from os.path import isfile, join
import datetime as dt
from datetime import datetime, timedelta

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f



In [3]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"

In [8]:
ISOS = [file for file in os.listdir(FILE_PATH) if file[0]!='.']
ISOS.remove("ISOs")
print(ISOS)

['SPP', 'NYISO', 'CAISO', 'PJM', 'MISO', 'ERCOT', 'ISONE']


In [9]:
ISO_files = {}
ISO_data = {}
ISO_final = {}

#load in
for ISO in ISOS:
    ISO_files[ISO] = listdir_nohidden(f"{FILE_PATH}/{ISO}/old")
    ISO_data[ISO] = {}
    for file in tqdm(ISO_files[ISO]):
        compiled = re.compile(re.escape(f"{ISO}_"), re.IGNORECASE)

        ISO_data[ISO][compiled.sub("", file).replace(".csv","")] = pd.read_csv(f"{FILE_PATH}/{ISO}/old/{file}")

13it [00:00, 18.13it/s]
14it [00:01,  9.22it/s]
13it [00:01, 11.79it/s]
16it [00:03,  4.10it/s]
16it [00:00, 17.95it/s]
13it [00:01, 10.50it/s]
19it [00:01, 10.41it/s]


In [10]:
#add merge datetime
titles = {}
for key in ISO_data:
    print(key)
    titles[key]= []
    for k in ISO_data[key]:
        try:
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
        except:
            ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
            print(type(ISO_data[key][k]["time_tomerge"]))

SPP
NYISO
CAISO
PJM
MISO
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
ERCOT
ISONE


In [11]:
ISO_final = {}
def clean_final(ISO: str, df: str):
    ISO_data[ISO][df].columns = ISO_data[ISO][df].columns.str.lower()
    no_date = [col for col in ISO_data[ISO][df].columns if "date" not in col]
    ISO_data[ISO][df] = ISO_data[ISO][df][no_date]
    try:
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("unnamed: 0")
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("hour")
    except:
        pass
    ISO_data[ISO][df] = ISO_data[ISO][df].sort_values(by="time_tomerge")
    ISO_data[ISO][df] = ISO_data[ISO][df].set_index("time_tomerge")
    ISO_final[ISO] = ISO_data[ISO][df]
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [12]:
clean_final("ISONE", "all_data")
clean_final("ERCOT", "final")
clean_final("MISO", "final")
clean_final("PJM", "final")
clean_final("CAISO", "all_data")
clean_final("NYISO", "all_data")
clean_final("SPP", "final")



                           congestion_da  congestion_rt  energy_da  energy_rt  \
time_tomerge                                                                    
2014-01-01 05:00:00+00:00            0.0        0.00000      62.04     117.43   
2014-01-01 06:00:00+00:00            0.0        0.00000      54.65     148.38   
2014-01-01 07:00:00+00:00            0.0        0.00000      38.40      97.06   
2014-01-01 08:00:00+00:00            0.0        0.00000      38.02      38.54   
2014-01-01 09:00:00+00:00            0.0        0.00000      37.48      79.01   
...                                  ...            ...        ...        ...   
2019-01-01 00:00:00+00:00            0.0       -0.01875      69.19      24.96   
2019-01-01 01:00:00+00:00            0.0       -0.01125      68.83      21.33   
2019-01-01 02:00:00+00:00            0.0        0.00000      59.21      22.37   
2019-01-01 03:00:00+00:00            0.0        0.00000      53.27      24.91   
2019-01-01 04:00:00+00:00   

In [40]:
ISO_done = ["NYISO"]
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"
new_data = {}
for ISO in ISO_done:
    sub_folder = f"{FILE_PATH}/{ISO}/new/combined"
    files = [f for f in listdir(sub_folder) if isfile(join(sub_folder, f)) and f[0]!="."]
    new_data[ISO] = {}
    print(ISO)
    for file in tqdm(files):
        new_data[ISO][file.replace(".csv", "")] = pd.read_csv(f"{sub_folder}/{file}")

NYISO


100%|██████████| 6/6 [00:20<00:00,  3.49s/it]


In [14]:
new_data[ISO]

{'ancillary_da':         Unnamed: 0    Eastern Date Hour Eastern Date Hour Time Zone Zone Name  \
 0                0  2019/01/01 00:00:00                         EST    CAPITL   
 1               10  2019/01/01 00:00:00                         EST      WEST   
 2                9  2019/01/01 00:00:00                         EST     NORTH   
 3                8  2019/01/01 00:00:00                         EST    N.Y.C.   
 4                7  2019/01/01 00:00:00                         EST    MILLWD   
 ...            ...                  ...                         ...       ...   
 377779       48581  2022/12/31 23:00:00                         EST    LONGIL   
 377780       48582  2022/12/31 23:00:00                         EST    MHK VL   
 377781       48583  2022/12/31 23:00:00                         EST    MILLWD   
 377782       48584  2022/12/31 23:00:00                         EST    N.Y.C.   
 377783       48586  2022/12/31 23:00:00                         EST      WEST   


In [15]:
[key for key in new_data[ISO]]

['ancillary_da',
 'energy_da',
 'ancillary_rt',
 'energy_rt',
 'generation',
 'load']

In [17]:
[key for key in ISO_final[ISO]]

['da_10sr',
 'da_10nsr',
 'da_30or',
 'da_regcap',
 'rt_10sr',
 'rt_10nsr',
 'rt_30or',
 'rt_regcap',
 'rt_regmov',
 'congestion_da',
 'congestion_rt',
 'energy_da',
 'energy_rt',
 'exchange',
 'unnamed: 0',
 'gas_price',
 'gen_dual_fuel',
 'gen_hydro',
 'gen_gas',
 'gen_nuclear',
 'gen_other_fossil',
 'gen_other_renewables',
 'gen_wind',
 'load',
 'load_forecast',
 'losses_da',
 'losses_rt']

In [32]:
#ancillary
print(np.unique(new_data[ISO]["ancillary_rt"].columns))
print(new_data[ISO]["ancillary_rt"])

['RTD 10 Min Non Sync' 'RTD 10 Min Sync' 'RTD 30 Min Non Sync'
 'RTD AS Price Version' 'RTD End Time Stamp'
 'RTD NYCA Regulation Capacity' 'RTD NYCA Regulation Movement'
 'Unnamed: 0' 'Zone Name' 'Zone PTID' 'time_tomerge']
         Unnamed: 0   RTD End Time Stamp Zone Name  Zone PTID  \
0                 0  2019/01/01 00:05:00    CAPITL      61757   
1                10  2019/01/01 00:05:00      WEST      61752   
2                 9  2019/01/01 00:05:00     NORTH      61755   
3                 8  2019/01/01 00:05:00    N.Y.C.      61761   
4                 7  2019/01/01 00:05:00    MILLWD      61759   
...             ...                  ...       ...        ...   
4721503      598183  2023/01/01 00:00:00    GENESE      61753   
4721504      598182  2023/01/01 00:00:00    DUNWOD      61760   
4721505      598181  2023/01/01 00:00:00    CENTRL      61754   
4721506      598186  2023/01/01 00:00:00    MHK VL      61756   
4721507      598180  2023/01/01 00:00:00    CAPITL      6175

In [43]:
print(type(new_data[ISO]['ancillary_da']["time_tomerge"][0]))
new_data[ISO]['ancillary_da']["time_tomerge"] = pd.to_datetime(new_data[ISO]['ancillary_da']["time_tomerge"], utc = True)
ancillary_da = pd.DataFrame(new_data[ISO]['ancillary_da'].groupby(['time_tomerge'])['DAM 10 Min Non Sync', 'DAM 10 Min Sync', 'DAM 30 Min Non Sync', 'DAM NYCA Regulation Capacity'].mean())
ancillary_da = ancillary_da.rename(columns = {'DAM 10 Min Non Sync': 'da_10nsr', 'DAM 10 Min Sync': 'da_10sr', 'DAM 30 Min Non Sync': "da_30or", 'DAM NYCA Regulation Capacity': 'da_regcap'})
print(ancillary_da)

<class 'str'>
                           da_10nsr   da_10sr   da_30or  da_regcap
time_tomerge                                                      
2019-01-01 00:00:00+00:00  3.000000  3.000000  3.000000      26.77
2019-01-01 01:00:00+00:00  3.000000  3.000000  3.000000      21.10
2019-01-01 02:00:00+00:00  3.000000  3.000000  3.000000      21.10
2019-01-01 03:00:00+00:00  3.000000  3.000000  3.000000       3.00
2019-01-01 04:00:00+00:00  3.000000  3.000000  3.000000      21.10
...                             ...       ...       ...        ...
2022-12-31 19:00:00+00:00  6.074545  6.074545  4.983636      15.21
2022-12-31 20:00:00+00:00  5.381818  5.381818  4.700000       5.95
2022-12-31 21:00:00+00:00  5.381818  5.381818  4.700000       8.87
2022-12-31 22:00:00+00:00  6.422727  6.422727  5.850000       6.90
2022-12-31 23:00:00+00:00  6.545455  6.545455  6.000000      12.90

[34340 rows x 4 columns]


/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_43952/3158824336.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ancillary_da = pd.DataFrame(new_data[ISO]['ancillary_da'].groupby(['time_tomerge'])['DAM 10 Min Non Sync', 'DAM 10 Min Sync', 'DAM 30 Min Non Sync', 'DAM NYCA Regulation Capacity'].mean())


In [41]:

print(type(new_data[ISO]['ancillary_rt']["time_tomerge"][0]))
new_data[ISO]['ancillary_rt']["time_tomerge"] = pd.to_datetime(new_data[ISO]['ancillary_rt']["time_tomerge"], utc = True).dt.floor("H")
#new_data[ISO]['ancillary_rt']["time_tomerge"] = pd.to_datetime(new_data[ISO]['ancillary_rt']["time_tomerge"])-timedelta(hours=5)
ancillary_rt = pd.DataFrame(new_data[ISO]['ancillary_rt'].groupby(['time_tomerge'])['RTD 10 Min Non Sync', 'RTD 10 Min Sync', 'RTD 30 Min Non Sync', 'RTD NYCA Regulation Capacity', 'RTD NYCA Regulation Movement'].mean())
ancillary_rt = ancillary_rt.rename(columns = {'RTD 10 Min Non Sync': 'rt_10nsr', 'RTD 10 Min Sync': 'rt_10sr', 'RTD 30 Min Non Sync': "da_30or", 'RTD NYCA Regulation Capacity': 'da_regcap', 'RTD NYCA Regulation Movement': 'rt_regmov'})
print(ancillary_rt)

<class 'str'>


/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_43952/3001067143.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ancillary_rt = pd.DataFrame(new_data[ISO]['ancillary_rt'].groupby(['time_tomerge'])['RTD 10 Min Non Sync', 'RTD 10 Min Sync', 'RTD 30 Min Non Sync', 'RTD NYCA Regulation Capacity', 'RTD NYCA Regulation Movement'].mean())


                           rt_10nsr   rt_10sr  da_30or  da_regcap  rt_regmov
time_tomerge                                                                
2019-01-01 00:00:00+00:00       0.0  2.825035      0.0  26.334615   0.250000
2019-01-01 01:00:00+00:00       0.0  0.000000      0.0  21.100000   0.300000
2019-01-01 02:00:00+00:00       0.0  0.000000      0.0  21.100000   0.300000
2019-01-01 03:00:00+00:00       0.0  0.000000      0.0   0.000000   0.300000
2019-01-01 04:00:00+00:00       0.0  0.000000      0.0  17.583333   0.300000
...                             ...       ...      ...        ...        ...
2022-12-31 20:00:00+00:00       0.0  0.000000      0.0   5.058333   0.154167
2022-12-31 21:00:00+00:00       0.0  0.000000      0.0  14.181667   0.361667
2022-12-31 22:00:00+00:00       0.0  0.000000      0.0   7.751667   0.191667
2022-12-31 23:00:00+00:00       0.0  0.000000      0.0   8.290000   0.109167
2023-01-01 00:00:00+00:00       0.0  0.000000      0.0  20.700000   0.060000

In [52]:
l1 = list(ancillary_da.index)
l2 = list(ancillary_rt.index)
missing = []
for l in tqdm(range(len(l2))):
    if l2[l] not in l1:
        missing.append(l2[l])
missing

100%|██████████| 35061/35061 [00:47<00:00, 731.35it/s] 


[Timestamp('2021-12-02 00:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 01:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 02:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 03:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 04:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 05:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 06:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 07:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 08:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 09:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 10:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 11:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 12:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 13:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 14:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 15:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 16:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 17:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 18:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 19:00:00+0000', tz='UTC'),


In [61]:
#new_data[ISO]["energy_rt"] = new_data[ISO]["energy_rt"].drop(columns=["Unnamed: 0", 'Unnamed: 0.1'])
new_data[ISO]["energy_rt"]["time_tomerge"] = pd.to_datetime(new_data[ISO]["energy_rt"]["Time"], utc=True)
new_data[ISO]["energy_rt"] = new_data[ISO]["energy_rt"].sort_values(by="time_tomerge")

In [71]:
subset = new_data[ISO]["energy_rt"]
subset["time_tomerge"] = subset["time_tomerge"].dt.floor("H")
subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())
subset = subset.drop_duplicates().rename(columns={'Energy':'energy_rt','Congestion':'congestion_rt', "Loss": "losses_rt"})
print(subset[subset.index.year !=2018])
energy_rt = subset[subset.index.year !=2018]

/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_43952/899610445.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())


                           energy_rt  congestion_rt  losses_rt
time_tomerge                                                  
2019-01-01 00:00:00+00:00   6.529222     -10.054778   0.251278
2019-01-01 01:00:00+00:00   6.486389     -10.135278   0.251611
2019-01-01 02:00:00+00:00   8.581889     -10.675500   0.313000
2019-01-01 03:00:00+00:00  -2.484833     -18.534000  -0.089944
2019-01-01 04:00:00+00:00   7.419889      -6.509278   0.229944
...                              ...            ...        ...
2023-01-01 01:00:00+00:00  30.349611       0.816389   0.801000
2023-01-01 02:00:00+00:00  25.640444       0.538000   0.609333
2023-01-01 03:00:00+00:00  26.697000       0.000389   0.645944
2023-01-01 04:00:00+00:00  25.833222      -0.984333   0.647000
2023-01-01 05:00:00+00:00  26.270667      -5.906000   0.651333

[35070 rows x 3 columns]


In [72]:
#new_data[ISO]["energy_rt"] = new_data[ISO]["energy_rt"].drop(columns=["Unnamed: 0", 'Unnamed: 0.1'])
new_data[ISO]["energy_da"]["time_tomerge"] = pd.to_datetime(new_data[ISO]["energy_da"]["Time"], utc=True)
new_data[ISO]["energy_da"] = new_data[ISO]["energy_da"].sort_values(by="time_tomerge")

In [73]:
subset = new_data[ISO]["energy_da"]
subset["time_tomerge"] = subset["time_tomerge"].dt.floor("H")
subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())
subset = subset.drop_duplicates().rename(columns={'Energy':'energy_da','Congestion':'congestion_da', "Loss": "losses_da"})
print(subset[subset.index.year !=2018])
energy_da =subset[subset.index.year !=2018]

/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_43952/2212635813.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())


                           energy_da  congestion_da  losses_da
time_tomerge                                                  
2019-01-01 00:00:00+00:00  20.642000      -8.138667   0.965333
2019-01-01 01:00:00+00:00  19.760667      -5.241333   0.911333
2019-01-01 02:00:00+00:00  19.378667      -3.818667   0.861333
2019-01-01 03:00:00+00:00  16.746667      -4.753333   0.650667
2019-01-01 04:00:00+00:00  16.675333      -4.318000   0.592000
...                              ...            ...        ...
2023-01-01 00:00:00+00:00  32.485333      -5.516667   0.883333
2023-01-01 01:00:00+00:00  32.455333      -2.342667   0.886000
2023-01-01 02:00:00+00:00  30.455333      -2.824000   0.740667
2023-01-01 03:00:00+00:00  28.338000      -2.798667   0.720667
2023-01-01 04:00:00+00:00  21.406667      -8.022000   0.548667

[35069 rows x 3 columns]


In [75]:
print(new_data[ISO]["generation"])

        Unnamed: 0  Unnamed: 0.1                       Time  Dual Fuel  \
0                0             0  2018-01-01 00:05:00-05:00     4619.0   
1                0             0  2022-01-01 00:05:00-05:00     2811.0   
2                0             0  2020-01-01 00:05:00-05:00     2350.0   
3                0             0  2019-01-01 00:05:00-05:00     1872.0   
4                0             0  2021-01-01 00:05:00-05:00     3155.0   
...            ...           ...                        ...        ...   
534719      108039        108039  2022-12-31 23:40:00-05:00     2555.0   
534720      108040        108040  2022-12-31 23:45:00-05:00     2531.0   
534721      108041        108041  2022-12-31 23:50:00-05:00     2511.0   
534722      108042        108042  2022-12-31 23:55:00-05:00     2485.0   
534723      108043        108043  2023-01-01 00:00:00-05:00     2461.0   

         Hydro  Natural Gas  Nuclear  Other Fossil Fuels  Other Renewables  \
0       2607.0       2700.0   541

In [78]:
new_data[ISO]["generation"]["time_tomerge"] = pd.to_datetime(new_data[ISO]["generation"]["Time"], utc=True)
new_data[ISO]["generation"] = new_data[ISO]["generation"].sort_values(by="time_tomerge")
print(new_data[ISO]["generation"].columns)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Time', 'Dual Fuel', 'Hydro',
       'Natural Gas', 'Nuclear', 'Other Fossil Fuels', 'Other Renewables',
       'Wind', 'time_tomerge'],
      dtype='object')


In [80]:
subset = new_data[ISO]["generation"]
subset["time_tomerge"] = subset["time_tomerge"].dt.floor("H")
subset = pd.DataFrame(subset.groupby(["time_tomerge"])['Dual Fuel', 'Hydro', 'Natural Gas', 'Nuclear', 'Other Fossil Fuels', 'Other Renewables', 'Wind'].mean())
subset = subset.drop_duplicates().rename(columns={'Dual Fuel': 'gen_dual_fuel', 'Hydro': 'gen_hydro', 'Natural Gas': 'gen_gas', 'Nuclear': 'gen_nuclear', 'Other Fossil Fuels': 'gen_other_fossil', 'Other Renewables': 'gen_other_renewables', 'Wind': 'gen_wind'})
print(subset[subset.index.year !=2018])
generation =subset[subset.index.year !=2018]

                           gen_dual_fuel    gen_hydro      gen_gas  \
time_tomerge                                                         
2019-01-01 00:00:00+00:00    3728.916667  3807.333333  1996.500000   
2019-01-01 01:00:00+00:00    3144.750000  3788.500000  1871.833333   
2019-01-01 02:00:00+00:00    2979.750000  3649.666667  1629.000000   
2019-01-01 03:00:00+00:00    2826.500000  3286.333333  1478.250000   
2019-01-01 04:00:00+00:00    2209.833333  3004.916667  1372.083333   
...                                  ...          ...          ...   
2023-01-01 01:00:00+00:00    3067.500000  3413.083333  3633.333333   
2023-01-01 02:00:00+00:00    2713.083333  3412.750000  3269.500000   
2023-01-01 03:00:00+00:00    2619.416667  3214.916667  2969.583333   
2023-01-01 04:00:00+00:00    2539.000000  2897.333333  2708.583333   
2023-01-01 05:00:00+00:00    2461.000000  2841.000000  2666.000000   

                           gen_nuclear  gen_other_fossil  \
time_tomerge                 

/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_43952/2763448192.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  subset = pd.DataFrame(subset.groupby(["time_tomerge"])['Dual Fuel', 'Hydro', 'Natural Gas', 'Nuclear', 'Other Fossil Fuels', 'Other Renewables', 'Wind'].mean())


In [76]:
[key for key in ISO_final[ISO]]


['da_10sr',
 'da_10nsr',
 'da_30or',
 'da_regcap',
 'rt_10sr',
 'rt_10nsr',
 'rt_30or',
 'rt_regcap',
 'rt_regmov',
 'congestion_da',
 'congestion_rt',
 'energy_da',
 'energy_rt',
 'exchange',
 'unnamed: 0',
 'gas_price',
 'gen_dual_fuel',
 'gen_hydro',
 'gen_gas',
 'gen_nuclear',
 'gen_other_fossil',
 'gen_other_renewables',
 'gen_wind',
 'load',
 'load_forecast',
 'losses_da',
 'losses_rt']

In [82]:
print(new_data[ISO]["load"])

        Unnamed: 0           Time Stamp Time Zone    Name   PTID  \
0               10  01/01/2019 00:00:00       EST    WEST  61752   
1                0  01/01/2019 00:00:00       EST  CAPITL  61757   
2                1  01/01/2019 00:00:00       EST  CENTRL  61754   
3                2  01/01/2019 00:00:00       EST  DUNWOD  61760   
4                3  01/01/2019 00:00:00       EST  GENESE  61753   
...            ...                  ...       ...     ...    ...   
385699         262  12/31/2022 23:00:00       EST   NORTH  61755   
385700         261  12/31/2022 23:00:00       EST  N.Y.C.  61761   
385701         257  12/31/2022 23:00:00       EST  HUD VL  61758   
385702         258  12/31/2022 23:00:00       EST  LONGIL  61762   
385703         260  12/31/2022 23:00:00       EST  MILLWD  61759   

        Integrated Load         time_tomerge  
0             1443.2000  2019-01-01 00:00:00  
1             1131.1000  2019-01-01 00:00:00  
2             1597.6000  2019-01-01 00:00:

In [ ]:
def create_date(entry: tuple):
    return pd.to_datetime(pd.to_datetime(entry[0]) + timedelta(hours = int(entry[1])))

def add_caiso_ancillary(sample_data: pd.DataFrame, ANC_TYPE: str, ANC_REGION: str, RESULT_TYPE: str):
    subset = sample_data[(sample_data["ANC_TYPE"] == ANC_TYPE[13:].upper()) & (sample_data["ANC_REGION"] == ANC_REGION) & (sample_data["RESULT_TYPE"] == RESULT_TYPE)]
    subset = subset[subset["OPR_HR"]<25]
    subset = pd.DataFrame(subset.groupby(['OPR_DT', "OPR_HR"])["MW"].mean())
    subset["DATE_HR"] = subset.index.to_numpy()
    subset["time_tomerge"] = subset["DATE_HR"].apply(create_date)
    #subset["time_tomerge"] = subset["time_tomerge"].map(lambda x: x.tz_convert('US/Pacific'))
    subset = subset.drop(columns = ["DATE_HR"])
    subset = subset.rename(columns={"MW": ANC_TYPE})
    subset = subset.drop_duplicates()
    print(len(np.unique(subset["time_tomerge"])))
    print(len(subset["time_tomerge"]))

    return subset